In [40]:
import pandas as pd


In [ ]:
df = pd.read_csv('fake_news/train.csv')


In [51]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [52]:
df.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [53]:
df = df.dropna()

In [55]:
X = df.drop('label' , axis = 1)
y= df['label']

In [56]:
import tensorflow as tf


In [57]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [58]:
voc_size = 5000

In [59]:
messages = X.copy()

In [60]:
messages.reset_index(inplace = True)

In [61]:
import nltk
import re
from nltk.corpus import stopwords

In [62]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [64]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [65]:
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [66]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [67]:
one_hot_repr = [one_hot(words, voc_size) for words in corpus]
one_hot_repr

[[93, 3550, 2556, 4934, 3973, 560, 4946, 3053, 2986, 4972],
 [4183, 1292, 4496, 1002, 4567, 1890, 188],
 [712, 1980, 4426, 860],
 [821, 1629, 515, 4348, 2026, 3473],
 [2574, 4567, 4468, 3083, 2289, 784, 4567, 702, 3943, 3400],
 [2379,
  930,
  4027,
  996,
  2558,
  4075,
  3077,
  2168,
  1027,
  3639,
  1972,
  4371,
  4139,
  4973,
  188],
 [4809, 233, 1086, 3252, 1479, 1269, 3204, 3561, 194, 3699, 1731],
 [4461, 3098, 1137, 378, 2976, 470, 4075, 2085, 194, 3699, 1731],
 [4340, 779, 212, 3253, 3780, 1892, 3087, 3893, 4075, 4911],
 [4633, 945, 117, 1143, 3237, 681, 2480, 1742],
 [2082, 857, 507, 729, 2317, 2286, 870, 2679, 4614, 1695, 1090],
 [4348, 4587, 3973, 1892, 4075, 2976],
 [1349, 477, 4417, 532, 3580, 2096, 4498, 4751, 4969],
 [3017, 685, 4887, 4688, 1501, 760, 3182, 194, 3699, 1731],
 [2738, 3655, 4573, 2210, 4856, 194, 3699, 1731],
 [1410, 3355, 3761, 4603, 951, 4127, 700, 3321, 2277, 4283],
 [311, 4371, 1292],
 [3394, 99, 3950, 2685, 4075, 2884, 216, 188],
 [1341, 3017, 44

In [68]:
sent_length = 20
embedded_docs = pad_sequences(one_hot_repr , padding = 'pre' , maxlen = sent_length)
print(embedded_docs)

[[   0    0    0 ... 3053 2986 4972]
 [   0    0    0 ... 4567 1890  188]
 [   0    0    0 ... 1980 4426  860]
 ...
 [   0    0    0 ...  194 3699 1731]
 [   0    0    0 ... 4140 2008 2356]
 [   0    0    0 ...  502  711 1027]]


In [69]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features , input_length = sent_length))
model.add(LSTM(100))
model.add(Dense(1 , activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'])
print(model.summary())

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [70]:
len(embedded_docs),y.shape

(18285, (18285,))

In [71]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [72]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [73]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [74]:
model.fit(X_train , y_train , validation_data = (X_test , y_test) , epochs = 10 , batch_size = 64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.7937 - loss: 0.4234 - val_accuracy: 0.9133 - val_loss: 0.1970
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9436 - loss: 0.1352 - val_accuracy: 0.9195 - val_loss: 0.2061
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9673 - loss: 0.0896 - val_accuracy: 0.9165 - val_loss: 0.2344
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9759 - loss: 0.0683 - val_accuracy: 0.9157 - val_loss: 0.2822
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9870 - loss: 0.0417 - val_accuracy: 0.9065 - val_loss: 0.3105
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9915 - loss: 0.0307 - val_accuracy: 0.9112 - val_loss: 0.4093
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9953 - loss: 0.0164 - val_accuracy: 0.9105 - val_loss: 0.4868
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9983 - loss: 0.0079 - val_accuracy: 0

In [76]:
from sklearn.metrics import classification_report , accuracy_score , confusion_matrix

y_pred = model.predict(X_test)

189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [77]:
y_pred = np.where(y_pred >0.5 , 1, 0)

In [78]:
confusion_matrix(y_test , y_pred)

array([[3174,  245],
       [ 329, 2287]])

In [79]:
accuracy_score(y_test , y_pred)

0.9048881524440763

In [82]:
print(classification_report(y_test , y_pred))

              precision    recall  f1-score   support

           0       0.91      0.93      0.92      3419
           1       0.90      0.87      0.89      2616

    accuracy                           0.90      6035
   macro avg       0.90      0.90      0.90      6035
weighted avg       0.90      0.90      0.90      6035

